# Load the data

In [19]:
import numpy as np
import pandas as pd
import os

In [20]:
PROJ_ROOT = "./"

In [21]:
!ls {PROJ_ROOT}

921_data.fth               real_samples.index.npy
921_etd.fth                sample_submission.csv
Untitled.ipynb             santendar-Copy1.ipynb
X_aug_df.feather           santendar.ipynb
X_train.csv                sub_2.csv
X_train_aug.feather        sub_3.csv
create-data.ipynb          test.csv
lgb-2-leaves-augment.ipynb train.csv
lgb_all_predictions.csv    xt.csv
lgb_oof.csv                y_aug_df.feather
lgb_submission.csv         y_train_aug.feather
lgbm_importances.png


In [22]:
TRAIN_DATA = os.path.join(PROJ_ROOT, "train.csv")
TEST_DATA = os.path.join(PROJ_ROOT, "test.csv")
SAMPLE_SUB = os.path.join(PROJ_ROOT, "sample_submission.csv")

## Read train data

In [23]:
%%time
train_df = pd.read_csv(TRAIN_DATA,index_col = 0)

CPU times: user 6.6 s, sys: 548 ms, total: 7.15 s
Wall time: 7.17 s


In [24]:
train_df.head()

,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
ID_code,,,,,,,,,,,,,,,,,,,,,
train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,-4.9200,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,3.1468,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,-4.9193,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,-5.8609,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,6.2654,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


## Read test data

In [25]:
%%time
test_df = pd.read_csv(TEST_DATA, index_col = 0)
print("number of test samples = %d "%(len(test_df)))

test_df.head()

number of test samples = 200000 
CPU times: user 6.46 s, sys: 499 ms, total: 6.95 s
Wall time: 6.97 s


# Get X & y

In [26]:
target = 'target'
features = [ col for col in train_df.columns if col not in target ]

X = train_df[features]
y = train_df[target]
X.head(), y.head()

(           var_0   var_1    var_2   var_3    var_4   var_5   var_6    var_7  \
 ID_code                                                                       
 train_0   8.9255 -6.7863  11.9081  5.0930  11.4607 -9.2834  5.1187  18.6266   
 train_1  11.5006 -4.1473  13.8588  5.3890  12.3622  7.0433  5.6208  16.5338   
 train_2   8.6093 -2.7457  12.0805  7.8928  10.5825 -9.0837  6.9427  14.6155   
 train_3  11.0604 -2.1518   8.9522  7.1957  12.5846 -1.8361  5.8428  14.9250   
 train_4   9.8369 -1.4834  12.8746  6.6375  12.2772  2.4486  5.9405  19.2514   
 
           var_8   var_9  ...  var_190  var_191  var_192  var_193  var_194  \
 ID_code                  ...                                                
 train_0 -4.9200  5.7470  ...   4.4354   3.9642   3.1364   1.6910  18.5227   
 train_1  3.1468  8.0851  ...   7.6421   7.7214   2.5837  10.9516  15.4305   
 train_2 -4.9193  5.9525  ...   2.9057   9.7905   1.6704   1.6858  21.6042   
 train_3 -5.8609  8.2450  ...   4.4666   4.7433 

# Import required libraries

In [154]:


from matplotlib import pyplot as plt

from tqdm import tqdm_notebook as tqdm
from sklearn.preprocessing import MinMaxScaler


from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import StratifiedKFold


from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier 
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from lightgbm import LGBMModel
from xgboost import XGBClassifier



from sklearn.metrics import make_scorer
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import auc
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score 
from sklearn.metrics import precision_score


from keras.layers import Dense, Dropout, Activation
from keras.models import Sequential
import tensorflow as tf

random_state = 42
np.random.seed(random_state)


# Data augumentation

In [28]:

# Ref - https://www.kaggle.com/kernels/scriptcontent/11630101/download
def augment(x,y,t=2):
    xs,xn = [],[]
    for i in range(t):
        mask = y>0
        x1 = x[mask].copy()
        ids = np.arange(x1.shape[0])
        for c in range(x1.shape[1]):
            np.random.shuffle(ids)
            x1[:,c] = x1[ids][:,c]
        xs.append(x1)

    for i in range(t//2):
        mask = y==0
        x1 = x[mask].copy()
        ids = np.arange(x1.shape[0])
        for c in range(x1.shape[1]):
            np.random.shuffle(ids)
            x1[:,c] = x1[ids][:,c]
        xn.append(x1)

    xs = np.vstack(xs)
    xn = np.vstack(xn)
    ys = np.ones(xs.shape[0])
    yn = np.zeros(xn.shape[0])
    x = np.vstack([x,xs,xn])
    y = np.concatenate([y,ys,yn])
    return x,y

 # Helper methods 
 ## Plot ROC curve 

In [29]:

def plot_roc_curve(y_test, y_pred):
    fpr, tpr, threshold = roc_curve(y_test, y_pred, pos_label = 1)
    print("fpr = %s ,tpr = %s , threshold = %s " %(str(fpr),str(tpr),str(threshold)))
    #print(confusion_matrix(y_test,y_pred))
    roc_auc = auc(fpr, tpr)
    plt.title('Receiver Operating Characteristic')
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    plt.scatter(fpr,tpr, s = 30, color = 'Red', marker = 'x')
    plt.legend(loc = 'lower right')
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()
    return roc_auc_score(y_test,y_pred)




## Evaluate the model 

In [30]:
def evaluate_model(y_test, y_pred):
    print("====> MODEL EVALUATION: ")
    print("Accuray score = %f " %(accuracy_score(y_test,y_pred)))
    
    print("Precision score = %f " %(precision_score(y_test, y_pred, pos_label = 1)))
    
    print("Confusion Matrix: ")
    print(confusion_matrix(y_test, y_pred, labels = [1,0]))
    
    print("TP FN \n FP TN")
    
    plot_roc_curve(y_test,y_pred)
    
    return 
    

## Train and test a specific model 

In [31]:
def train_and_test_model(model, X_train, X_test, y_train, y_test):
    
    print("Training model with %d samples: " %len(X_train))
    print(model)
    
    # Fit the train samples  
    model.fit(X_train, y_train)
    print("Training completed!")
    
    print("Predicting the target for %d test samples:" %(len(X_test)))
      
    # Predict the target for test samples 
    y_pred = model.predict(X_test)
    print("Prediction complete!")
    
    evaluate_model(y_test, y_pred)
    return y_pred
    

## Grid search with roc_auc scorer & predict with best model  

In [32]:

def get_grid_best_estimator(X_train, y_train,base_estimator, params, scorer = None):
    # inputs of Grid search 
    
    # create cross validation sets 
    #cv_sets = ShuffleSplit(n_splits = 10, test_size = 0.2, random_state = 50)
    cv_sets = StratifiedKFold(n_splits = 10, shuffle = False, random_state = 50)
    
    # Default scorer 
    if scorer is None: 
        scorer = make_scorer(roc_auc_score)
    
    # params
    
    # Grid search
    grid = GridSearchCV(base_estimator, params, scoring = scorer, cv = cv_sets )
    
    # fit 
    grid.fit(X_train,y_train)
    
    best_estimator  = grid.best_estimator_
    
    
    print("Best estimator : ")
    print(best_estimator)
    
    return best_estimator



def train_and_test_grid_search(X_train, X_test, y_train, y_test, base_estimator, params, scorer = None):
    
    
    best_estimator = get_grid_best_estimator(X_train, y_train, base_estimator, params, scorer)
    
    print("Predicting the target for %d test samples:" %(len(X_test)))
    y_pred = best_estimator.predict(X_test)
    
    print("Prediction complete!")
    
    evaluate_model(y_test, y_pred)
    return y_pred

# Stratified K fold

In [34]:
sk_folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)


# Light GBM 

In [182]:

def get_lgbm_obj():
    return LGBMModel( objective = "binary", 
                          scale_pos_weight = 1,
                          metric = "auc", 
                          boosting = 'gbdt', 
                          max_depth = -1, 
                          num_leaves = 13, 
                          learning_rate = 0.01, 
                          bagging_freq = 5, 
                          bagging_fraction = 0.4, 
                          feature_fraction = 0.05, 
                          min_data_in_leaf = 80, 
                          min_sum_heassian_in_leaf = 10, 
                          tree_learner = "serial", 
                          boost_from_average = False, 
                          bagging_seed = random_state, 
                          verbosity = 1, 
                          seed = random_state,
                         num_boost_round = 100000,
                         is_training_metric = True,
                         is_provide_training_metric=True)
                          

In [179]:
def get_vc(): 

    clf0 = get_lgbm_obj()
    clf1 = RandomForestClassifier()
    clf2 = ExtraTreesClassifier()
    clf3 = GradientBoostingClassifier()
    clf4 = LogisticRegression()
    vc = VotingClassifier([['lgbm',clf0],['lreg',clf4],['rf',clf1],['etc',clf2],['gbc',clf3]],voting = 'hard', weights = None, n_jobs = 16)
    return vc
    

In [183]:
val_aucs = []

predictions = pd.DataFrame(index=test_df.index)
predictions.reset_index(inplace=True)


for fold, (train_idx,valid_idx) in tqdm(enumerate(sk_folds.split(X,y))):
    X_train, y_train = None,None
    X_valid, y_valid = None,None
    
    X_train,y_train = X.iloc[train_idx],y.iloc[train_idx]
    X_valid,y_valid = X.iloc[valid_idx],y.iloc[valid_idx]
    #print(f"Fold {fold},{len(X_train)},{len(y_train)},{len(X_valid)},{len(y_valid)}")
    
    
    sub_iters = 5
    sub_y_pred, sub_y_valid_pred = 0,0
    for i in tqdm(range(sub_iters)):
        X_train_aug, y_train_aug = None, None
        X_train_aug_df, y_train_aug_df = None, None
        lgbm = None

        print("Augumentation for fold %d, sub_iter %d ..."%(fold,i), end="")
        X_train_aug, y_train_aug = augment(X_train.values,y_train.values)
        print("Done")

        X_train_aug_df = pd.DataFrame(X_train_aug,columns = features)
        y_train_aug_df = pd.DataFrame(y_train_aug,columns = [target])

        print("Fit start....", end="")
        lgbm = get_lgbm_obj()

#         lgbm.fit(X_train_aug_df, y_train_aug_df)
        
        lgbm.fit(X_train_aug_df, y_train_aug_df, 
             eval_set = [(X_valid,y_valid)],
             early_stopping_rounds = 3000,
             eval_metric = 'auc',
             verbose = 1000)

        print("Done")
    
        # Predict the target for validation set 
        y_valid_pred = lgbm.predict(X_valid)
        sub_y_valid_pred += y_valid_pred
        
        print("Prediction start......", end = "")
        # Predict the target for actual test set 
        y_pred = lgbm.predict(test_df[features])
        sub_y_pred += y_pred
        print("Done")
        # Save the predictions on the test data for this fold 
        
    #print(pd.DataFrame(sub_y_valid_pred).describe())
    #print(type(sub_y_valid_pred),sub_y_valid_pred.shape)
    val_score = roc_auc_score(y_valid,sub_y_valid_pred)
    print("fold %d, sub_iter %d ... AUC = %f "%(fold,i,val_score), end="")

    val_aucs.append(val_score)
    predictions[f"fold_{fold}"] = sub_y_pred/sub_iters

Augumentation for fold 0, sub_iter 0 ...Done
Fit start....

/Users/vthotakura/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:118: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 3000 rounds.
[1000]	valid_0's auc: 0.882241
[2000]	valid_0's auc: 0.889115
[3000]	valid_0's auc: 0.893536
[4000]	valid_0's auc: 0.896231
[5000]	valid_0's auc: 0.897942
[6000]	valid_0's auc: 0.898923
[7000]	valid_0's auc: 0.899498
[8000]	valid_0's auc: 0.899805
[9000]	valid_0's auc: 0.900015
[10000]	valid_0's auc: 0.900045
[11000]	valid_0's auc: 0.900063
[12000]	valid_0's auc: 0.899961
Early stopping, best iteration is:
[9475]	valid_0's auc: 0.90009
Done
Prediction start......Done
Augumentation for fold 0, sub_iter 1 ...Done
Fit start....Training until validation scores don't improve for 3000 rounds.
[1000]	valid_0's auc: 0.882373
[2000]	valid_0's auc: 0.88911
[3000]	valid_0's auc: 0.893281
[4000]	valid_0's auc: 0.89608
[5000]	valid_0's auc: 0.897851
[6000]	valid_0's auc: 0.898874
[7000]	valid_0's auc: 0.899662
[8000]	valid_0's auc: 0.9
[9000]	valid_0's auc: 0.900168
[10000]	valid_0's auc: 0.900259
[11000]	valid_0's auc: 0.900263
[12000

Augumentation for fold 1, sub_iter 0 ...Done
Fit start....

/Users/vthotakura/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:118: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


...

/Users/vthotakura/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:118: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 3000 rounds.
[1000]	valid_0's auc: 0.879577
[2000]	valid_0's auc: 0.886915
[3000]	valid_0's auc: 0.891655
[4000]	valid_0's auc: 0.894926
[5000]	valid_0's auc: 0.896881
[6000]	valid_0's auc: 0.898127
[7000]	valid_0's auc: 0.898968
[8000]	valid_0's auc: 0.899428
[9000]	valid_0's auc: 0.899651
[10000]	valid_0's auc: 0.899797
[11000]	valid_0's auc: 0.899784
[12000]	valid_0's auc: 0.899786
[13000]	valid_0's auc: 0.899741
[14000]	valid_0's auc: 0.899658
[15000]	valid_0's auc: 0.899519
Early stopping, best iteration is:
[12526]	valid_0's auc: 0.899849
Done
Prediction start......Done
Augumentation for fold 3, sub_iter 1 ...Done
Fit start....Training until validation scores don't improve for 3000 rounds.
[1000]	valid_0's auc: 0.879883
[2000]	valid_0's auc: 0.886946
[3000]	valid_0's auc: 0.891573
[4000]	valid_0's auc: 0.894703
[5000]	valid_0's auc: 0.896556
[6000]	valid_0's auc: 0.897805
[7000]	valid_0's auc: 0.898566
[8000]	valid_0's auc: 0.899

Augumentation for fold 4, sub_iter 0 ...Done
Fit start....

/Users/vthotakura/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:118: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 3000 rounds.
[1000]	valid_0's auc: 0.877416
[2000]	valid_0's auc: 0.884379
[3000]	valid_0's auc: 0.889076
[4000]	valid_0's auc: 0.892515
[5000]	valid_0's auc: 0.894546
[6000]	valid_0's auc: 0.89565
[7000]	valid_0's auc: 0.896361
[8000]	valid_0's auc: 0.896896
[9000]	valid_0's auc: 0.897128
[10000]	valid_0's auc: 0.897196
[11000]	valid_0's auc: 0.897216
[12000]	valid_0's auc: 0.897222
[13000]	valid_0's auc: 0.897115
[14000]	valid_0's auc: 0.897074
[15000]	valid_0's auc: 0.897124
Early stopping, best iteration is:
[12066]	valid_0's auc: 0.897242
Done
Prediction start......Done
Augumentation for fold 4, sub_iter 1 ...Done
Fit start....Training until validation scores don't improve for 3000 rounds.
[1000]	valid_0's auc: 0.876553
[2000]	valid_0's auc: 0.884149
[3000]	valid_0's auc: 0.889019
[4000]	valid_0's auc: 0.892193
[5000]	valid_0's auc: 0.89441
[6000]	valid_0's auc: 0.895605
[7000]	valid_0's auc: 0.896254
[8000]	valid_0's auc: 0.89675

In [205]:
#lgbm.evals_result_
val_aucs

[0.900684821079561,
 0.9014076597363921,
 0.9074235893904575,
 0.9003955018553478,
 0.8978753228999384]

In [184]:
from statistics import mean, median
np.mean(val_aucs)

0.9015573789923395

In [185]:
predictions.head()

,ID_code,fold_0,fold_1,fold_2,fold_3,fold_4
0,test_0,0.124144,0.117148,0.138225,0.122904,0.143871
1,test_1,0.346829,0.254465,0.281538,0.260350,0.282374
2,test_2,0.258258,0.251037,0.252986,0.251376,0.211798
3,test_3,0.247294,0.270641,0.289633,0.316084,0.292356
4,test_4,0.069074,0.043773,0.058764,0.062269,0.063083


In [186]:
for i in range(5):
    print(predictions[f"fold_{i}"].value_counts())

0.388138    1
0.148890    1
0.021690    1
0.052518    1
0.164718    1
0.005661    1
0.112228    1
0.145509    1
0.015664    1
0.335203    1
0.032309    1
0.004337    1
0.154786    1
0.003218    1
0.122651    1
0.009131    1
0.011033    1
0.449305    1
0.187470    1
0.051527    1
0.133893    1
0.012628    1
0.017994    1
0.013898    1
0.014022    1
0.015787    1
0.051454    1
0.010782    1
0.022258    1
0.304066    1
           ..
0.016639    1
0.027772    1
0.013514    1
0.104302    1
0.059238    1
0.560737    1
0.018899    1
0.019198    1
0.237282    1
0.033711    1
0.031808    1
0.070996    1
0.027679    1
0.048946    1
0.019299    1
0.024617    1
0.013541    1
0.009462    1
0.056734    1
0.014393    1
0.065911    1
0.150643    1
0.232771    1
0.024685    1
0.022038    1
0.689577    1
0.029732    1
0.065657    1
0.413173    1
0.038225    1
Name: fold_0, Length: 200000, dtype: int64
0.883329    1
0.016847    1
0.009011    1
0.372879    1
0.064518    1
0.109899    1
0.086154    1
0.078

In [187]:
predictions.columns

Index(['ID_code', 'fold_0', 'fold_1', 'fold_2', 'fold_3', 'fold_4'], dtype='object')

In [188]:
pred_cols = ['fold_0', 'fold_1', 'fold_2', 'fold_3', 'fold_4']

In [189]:
predictions['target'] = np.mean(predictions[[col for col in predictions.columns if col not in ['ID_code', 'target']]].values, axis=1)


In [190]:
predictions['target_round'] = predictions['target']

In [191]:
predictions['target_round'].value_counts()

0.014367    1
0.002760    1
0.005465    1
0.011138    1
0.028233    1
0.017371    1
0.086203    1
0.333004    1
0.035917    1
0.011653    1
0.072590    1
0.025942    1
0.221072    1
0.062079    1
0.032418    1
0.245434    1
0.056229    1
0.007341    1
0.075713    1
0.003169    1
0.019478    1
0.333497    1
0.006120    1
0.154048    1
0.123619    1
0.234412    1
0.095483    1
0.428377    1
0.043292    1
0.006516    1
           ..
0.430692    1
0.022688    1
0.048838    1
0.070517    1
0.004994    1
0.131559    1
0.036645    1
0.024668    1
0.039448    1
0.353982    1
0.033835    1
0.022090    1
0.477591    1
0.178083    1
0.013973    1
0.131084    1
0.043754    1
0.026067    1
0.006912    1
0.030715    1
0.110648    1
0.027581    1
0.128193    1
0.046229    1
0.162171    1
0.015243    1
0.006272    1
0.011619    1
0.016957    1
0.030087    1
Name: target_round, Length: 200000, dtype: int64

In [192]:
predictions['target'].value_counts()

0.014367    1
0.002760    1
0.005465    1
0.011138    1
0.028233    1
0.017371    1
0.086203    1
0.333004    1
0.035917    1
0.011653    1
0.072590    1
0.025942    1
0.221072    1
0.062079    1
0.032418    1
0.245434    1
0.056229    1
0.007341    1
0.075713    1
0.003169    1
0.019478    1
0.333497    1
0.006120    1
0.154048    1
0.123619    1
0.234412    1
0.095483    1
0.428377    1
0.043292    1
0.006516    1
           ..
0.430692    1
0.022688    1
0.048838    1
0.070517    1
0.004994    1
0.131559    1
0.036645    1
0.024668    1
0.039448    1
0.353982    1
0.033835    1
0.022090    1
0.477591    1
0.178083    1
0.013973    1
0.131084    1
0.043754    1
0.026067    1
0.006912    1
0.030715    1
0.110648    1
0.027581    1
0.128193    1
0.046229    1
0.162171    1
0.015243    1
0.006272    1
0.011619    1
0.016957    1
0.030087    1
Name: target, Length: 200000, dtype: int64

In [193]:
sample_sub = pd.read_csv(SAMPLE_SUB)

In [194]:
sample_sub.head()

,ID_code,target
0,test_0,0
1,test_1,0
2,test_2,0
3,test_3,0
4,test_4,0


In [195]:
sample_sub['target'] = predictions['target_round']

In [196]:
sample_sub.describe()

,target
count,200000.000000
mean,0.117253
std,0.168125
min,0.000093
25%,0.017912
50%,0.049545
75%,0.138007
max,0.998743


sample_sub['target

In [197]:
sample_sub.to_csv("./sub_final.csv",index=False)

In [199]:
sample_sub_round = sample_sub.copy()

In [200]:
predictions['target_round'] = np.(predictions['target'])

In [201]:
predictions.to_csv("./predictions_final.csv")

In [202]:
predictions.head()

,ID_code,fold_0,fold_1,fold_2,fold_3,fold_4,target,target_round
0,test_0,0.124144,0.117148,0.138225,0.122904,0.143871,0.129258,0.0
1,test_1,0.346829,0.254465,0.281538,0.260350,0.282374,0.285111,0.0
2,test_2,0.258258,0.251037,0.252986,0.251376,0.211798,0.245091,0.0
3,test_3,0.247294,0.270641,0.289633,0.316084,0.292356,0.283201,0.0
4,test_4,0.069074,0.043773,0.058764,0.062269,0.063083,0.059393,0.0


In [203]:
sample_sub_round['target'] = predictions['target_round']

In [204]:
sample_sub_round.to_csv("./sub_final_round.csv", index=False)